In [1]:
# 画像のサイズはMobileNetV2に対応した縦横96pxにリサイズする
import tensorflow as tf

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/train",
    image_size = (96,96),
    label_mode = "binary",
    batch_size = 32,
    shuffle = True
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/test",
    image_size = (96,96),
    label_mode = "binary",
    batch_size = 32,
    shuffle = False
)

Found 300 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


In [2]:
# 画像を水増しするための関数の定義
def flip_left_right(image, label):
    image = tf.image.flip_left_right(image)
    return image, label

def flip_up_down(image, label):
    image = tf.image.flip_up_down(image)
    return image, label

def rot90(image, label):
    image = tf.image.rot90(image)
    return image, label

def rot180(image, label):
    image = tf.image.rot90(image, k = 2)
    return image, label

def rot270(image, label):
    image = tf.image.rot90(image, k = 3)
    return image, label

# 画像の水増し処理を実行する
train_dataset_lr = train_dataset.map(flip_left_right)
train_dataset_ud = train_dataset.map(flip_up_down)
train_dataset_rot90 = train_dataset.map(rot90)
train_dataset_rot180 = train_dataset.map(rot180)
train_dataset_rot270 = train_dataset.map(rot270)

# 水増しした画像を訓練データに追加する
train_dataset = train_dataset.concatenate(train_dataset_lr)
train_dataset = train_dataset.concatenate(train_dataset_ud)
train_dataset = train_dataset.concatenate(train_dataset_rot90)
train_dataset = train_dataset.concatenate(train_dataset_rot180)
train_dataset = train_dataset.concatenate(train_dataset_rot270)

# 訓練データをシャッフルする
train_dataset = train_dataset.shuffle(32)

In [3]:
# MobileNetV2 モデルを作成する
input_layer = tf.keras.Input(shape = (96, 96, 3))
i_layer = tf.keras.applications.mobilenet_v2.preprocess_input(input_layer)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape = (96, 96, 3),
    input_tensor = i_layer,
    include_top = False,
    weights = "imagenet",
    pooling = 'avg'
)

base_model.trainable = False

# Dense層を追加する
output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')

In [4]:
# base_modelにoutput_layerを追加したモデルを作成する
model = tf.keras.Sequential([
    base_model,
    output_layer
])

# modelをcompileする
model.compile(
    optimizer = "adam",
    loss = 'binary_crossentropy',
    metrics = ["accuracy"]
)

# modelに学習をさせる
model.fit(train_dataset, epochs=20)

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.7817 - loss: 0.4587
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9244 - loss: 0.1917
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9483 - loss: 0.1378
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9672 - loss: 0.1069
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9756 - loss: 0.0872
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9817 - loss: 0.0751
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9833 - loss: 0.0656
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.9906 - loss: 0.0548
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.9900 - loss: 0.0498
Epoch 10/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.9939 - loss: 0.0425
Epoch 11/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.9967 - loss: 0.0383
Epoch 12/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy:

In [5]:
# evaluate()でモデルの性能を評価する
model.evaluate(test_dataset)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9800 - loss: 0.0435


[0.04352379962801933, 0.9800000190734863]